In [2]:
import pandas as pd
from vaderindo import vaderindo
data = pd.read_csv('testing_data.csv',index_col=None)

ModuleNotFoundError: No module named 'vaderindo.finvader'

In [ ]:
# use the classifier
import numpy as np
data['vaderindo'] = data.text.apply(vaderindo,use_sentibignomics = True, use_henry = True, indicator="compound")

In [ ]:
# discretize compound indicator into 3 scores
conditions = [
    (data['vaderindo'] > 0.33),
    (data['vaderindo']> - 0.33) & (data['vaderindo'] < 0.33),
    (data['vaderindo'] < - 0.33)
]
values = [1, 0, -1]
data['vaderindo'] = np.select(conditions, values)

In [ ]:
# calculate true positives (TP), true negatives (TN), false positives (FP) and false negatives (FN)
data['vaderindo_correct'] = np.where((data['label'] == data['vaderindo']), 1, 0)
data['vaderindo_TP'] = np.where((data['label'] == 1) & (data['vaderindo'] == 1), 1, 0)
data['vaderindo_TN'] = np.where((data['label'] == -1) & (data['vaderindo'] == -1), 1, 0)
data['vaderindo_FP'] = np.where((data['label'] != 1) & (data['vaderindo'] == 1), 1, 0)
data['vaderindo_FN'] = np.where((data['label'] != -1) & (data['vaderindo'] == -1), 1, 0)

In [ ]:
# print accuracy, precision, recall, F1 score
print("vaderindo accuracy is:",  round((((data['vaderindo_correct'].sum()) / len(data))) * 100, 2), "%")
print("vaderindo precision is:",  round((( data['vaderindo_TP'].sum() / ((data['vaderindo_TP'].sum() + data['vaderindo_FP'].sum())))) * 100, 2), "%")
print("vaderindo recall is:",  round((( data['vaderindo_TP'].sum() / ((data['vaderindo_TP'].sum() + data['vaderindo_FN'].sum())))) * 100, 2), "%")
print("vaderindo F1 score is:",  2 * round(
    (((data['vaderindo_TP'].sum() / ((data['vaderindo_TP'].sum() + data['vaderindo_FP'].sum()))) *
      (((data['vaderindo_TP'].sum() / ((data['vaderindo_TP'].sum() + data['vaderindo_FN'].sum())))))) /
     (((data['vaderindo_TP'].sum() / ((data['vaderindo_TP'].sum() + data['vaderindo_FP'].sum()))) +
       (((data['vaderindo_TP'].sum() / ((data['vaderindo_TP'].sum() + data['vaderindo_FN'].sum())))))))
     )* 100, 2), "%")

FinVADER accuracy is: 69.36 %
FinVADER precision is: 60.57 %
FinVADER recall is: 85.45 %
FinVADER F1 score is: 70.88 %


In [ ]:
# baseline VADER
# discretize compound indicator into 3 scores
from nltk.sentiment.vader import SentimentIntensityAnalyzer
baseline = SentimentIntensityAnalyzer()

def sentiment_vader(sentence):

    sentiment_dict = baseline.polarity_scores(sentence)

    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] > 0.33 :
        overall_sentiment = 1

    elif sentiment_dict['compound'] < - 0.33 :
        overall_sentiment = - 1

    else :
        overall_sentiment = 0

    return overall_sentiment

In [ ]:
# use the classifier
data['vader'] = data.text.apply(sentiment_vader)

In [ ]:
# calculate true positives (TP), true negatives (TN), false positives (FP) and false negatives (FN)
data['vader_correct'] = np.where((data['label'] == data['vader']), 1, 0)
data['vader_TP'] = np.where((data['label'] == 1) & (data['vader'] == 1), 1, 0)
data['vader_TN'] = np.where((data['label'] == -1) & (data['vader'] == -1), 1, 0)
data['vader_FP'] = np.where((data['label'] != 1) & (data['vader'] == 1), 1, 0)
data['vader_FN'] = np.where((data['label'] != -1) & (data['vader'] == -1), 1, 0)

In [ ]:
# print accuracy, precision, recall, F1 score
print("VADER accuracy is:",  round((((data['vader_correct'].sum()) / len(data))) * 100, 2), "%")
print("VADER precision is:",  round((( data['vader_TP'].sum() / ((data['vader_TP'].sum() + data['vader_FP'].sum())))) * 100, 2), "%")
print("VADER recall is:",  round((( data['vader_TP'].sum() / ((data['vader_TP'].sum() + data['vader_FN'].sum())))) * 100, 2), "%")
print("VADER F1 score is:",  2 * round(
    (((data['vader_TP'].sum() / ((data['vader_TP'].sum() + data['vader_FP'].sum()))) *
      (((data['vader_TP'].sum() / ((data['vader_TP'].sum() + data['vader_FN'].sum())))))) /
     (((data['vader_TP'].sum() / ((data['vader_TP'].sum() + data['vader_FP'].sum()))) +
       (((data['vader_TP'].sum() / ((data['vader_TP'].sum() + data['vader_FN'].sum())))))))
     )* 100, 2), "%")

VADER accuracy is: 58.75 %
VADER precision is: 44.41 %
VADER recall is: 88.28 %
VADER F1 score is: 59.1 %
